In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
def rearrange_sentences(sentences, length):
    """
    Rearranges a list of sentences into an array of complete sentences of equal lengths.

    Parameters:
    sentences (list of str): The list of sentences to rearrange.
    length (int): The desired length for each segment.

    Returns:
    list of str: An array of strings, each string containing sentences concatenated to form segments of the specified length.
    """
    result = []
    current_segment = ""
    
    for sentence in sentences:
        # Check if adding the current sentence exceeds the desired length
        if len(current_segment) + len(sentence) + 1 <= length:
            # If not, add the sentence to the current segment
            if current_segment:
                current_segment += " " + sentence
            else:
                current_segment = sentence
        else:
            # If it exceeds, add the current segment to the result and start a new segment
            result.append(current_segment)
            current_segment = sentence
    
    # Add the last segment to the result
    if current_segment:
        result.append(current_segment)
    
    return result

length = 150
result = rearrange_sentences(sentences, length)
for segment in result:
    print(segment)


Agent: Good morning! Thank you for calling XYZ Mobile Network. My name is Alex. How can I assist you today? Customer: Hi Alex, I'm Emma.
I have a few issues with my mobile network, and I was hoping you could help me out. Agent: Of course, Emma. I'm here to help.
Could you please describe the issues you're experiencing? Customer: Well, firstly, I've been having trouble with my signal.
It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars.
Agent: I'm sorry to hear that you're experiencing signal issues, Emma. Let's start by checking a few things.
Can you tell me where you usually face this problem? Is it a specific location like your home or office, or does it happen everywhere?
Customer: It's mostly at home. I live in an apartment on the 10th floor, and the signal is really unstable here. Agent: I understand.
Sometimes, signal issues can be due to building materials or interference. Can you try something for me?

In [73]:
# Sample conversations
conversations = [
    "Agent: Good morning! Thank you for calling XYZ Mobile Network.",
    "My name is Alex.",
    "How can I assist you today?",
    "Customer: Hi Alex, I'm Emma.",
    "I have a few issues with my mobile network, and I was hoping you could help me out.",
    "Agent: Of course, Emma.",
    "I'm here to help.",
    "Could you please describe the issues you're experiencing?",
    "Customer: Well, firstly, I've been having trouble with my signal.",
    "It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars.",
    "Agent: I'm sorry to hear that you're experiencing signal issues, Emma.",
    "Let's start by checking a few things.",
    "Can you tell me where you usually face this problem?",
    "Is it a specific location like your home or office, or does it happen everywhere?",
    "Customer: It's mostly at home.",
    "I live in an apartment on the 10th floor, and the signal is really unstable here.",
    "Agent: I understand.",
    "Sometimes, signal issues can be due to building materials or interference.",
    "Can you try something for me?",
    "If possible, switch your phone to airplane mode for a few seconds and then turn it back on.",
    "This can sometimes help reset the connection to the nearest tower.",
    "Customer: Okay, I just did that.",
    "Should I test a call now?",
    "Agent: Yes, please go ahead and try making a call.",
    "Let me know if there's any improvement.",
    "Customer: It seems to be working better now, but this is something I’ve had to do frequently.",
    "Is there a permanent solution?",
    "Agent: I can understand how frustrating that must be.",
    "There are a couple of options we can consider.",
    "One is to update your phone’s software, as updates can improve network connectivity.",
    "Another option is a network booster for your apartment, which amplifies the signal.",
    "Would you be interested in exploring either of these options?",
    "Customer: I’ve already updated my phone’s software.",
    "I’ve heard about network boosters but wasn’t sure if they were effective.",
    "Are they?",
    "Agent: Yes, network boosters can be very effective, especially in high-rise buildings where signals might struggle to penetrate.",
    "We offer a range of boosters that can cater to different needs.",
    "If you’d like, I can provide more details on the options available.",
    "Customer: That sounds promising.",
    "Please send me the details.",
    "Also, I’ve been noticing high charges on my bill for the past couple of months.",
    "I haven’t changed my usage patterns, so I’m confused.",
    "Agent: I’m sorry to hear about the billing issue, Emma.",
    "Let’s review your recent bills together.",
    "I’ll need to ask you some security questions first to verify your identity.",
    "Can you please confirm the last four digits of your Social Security number?",
    "Customer: Sure, it’s 6789.",
    "Agent: Thank you.",
    "I’ve pulled up your account.",
    "I can see there have been a few international calls and some data overages.",
    "Do you recall making these international calls or using more data than usual?",
    "Customer: Oh, I did make a few international calls, but I thought they were covered under my plan.",
    "As for the data, I’ve been using my Wi-Fi mostly.",
    "Agent: It looks like the international calls were made to countries not included in your plan’s coverage.",
    "I can recommend a couple of options: we have an add-on for international calls to specific countries at a lower rate.",
    "Regarding the data usage, sometimes apps run in the background and use mobile data even when Wi-Fi is available.",
    "I can guide you on how to monitor and manage your data usage.",
    "Customer: That makes sense.",
    "I’ll definitely look into the international calling add-on.",
    "How can I monitor my data usage?",
    "Agent: Most smartphones have a data usage monitor in the settings.",
    "You can set a data limit and receive alerts when you’re nearing it.",
    "Additionally, you can check which apps are using the most data.",
    "Would you like me to walk you through the steps on your specific phone model?",
    "Customer: Yes, please.",
    "I have an Android phone.",
    "Agent: Great!",
    "Here’s what you need to do: Go to your Settings, then select 'Network & Internet,' followed by 'Data usage.'",
    "Here, you’ll see a breakdown of your data usage by app.",
    "You can also set a data warning and limit by tapping on 'Data warning & limit.'",
    "This way, you’ll get notified before you exceed your limit.",
    "Customer: I see it now.",
    "This is really helpful.",
    "I’ll set it up right away.",
    "Thanks, Alex!",
    "Agent: You’re welcome, Emma.",
    "Is there anything else I can assist you with today?",
    "Customer: Actually, yes.",
    "I’m considering upgrading my plan.",
    "Can you tell me about the available options?",
    "Agent: Absolutely!",
    "We have several plans that offer more data, unlimited international calls, and additional perks like streaming service subscriptions.",
    "Based on your current usage, I’d recommend our Premium Plan.",
    "It includes unlimited data, free international calls to over 50 countries, and a subscription to a streaming service of your choice.",
    "Customer: That sounds perfect.",
    "How much would it cost to upgrade to the Premium Plan?",
    "Agent: The Premium Plan is $20 more per month than your current plan.",
    "However, with the included international calls and additional perks, it could save you money overall.",
    "Customer: That seems reasonable.",
    "I think I’ll go ahead with the upgrade.",
    "How do I proceed?",
    "Agent: I can handle the upgrade for you right now.",
    "Just to confirm, you’d like to switch to the Premium Plan at $20 more per month?",
    "Customer: Yes, that’s correct.",
    "Agent: Great!",
    "I’ve processed the upgrade.",
    "Your new plan is now active, and you can start enjoying the benefits immediately.",
    "Is there anything else I can do for you today?",
    "Customer: No, that’s all.",
    "Thanks a lot for your help, Alex.",
    "You’ve been fantastic!",
    "Agent: It’s been my pleasure, Emma.",
    "Thank you for choosing XYZ Mobile Network.",
    "Have a wonderful day!",
    "Customer: You too, Alex.",
    "Goodbye!"
]

conversations = rearrange_sentences(conversations, length)

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import string

# Download stopwords if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')


# Text preprocessing function
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return words

# Preprocess each conversation
processed_conversations = [preprocess(conversation) for conversation in conversations]

# Create a dictionary and corpus for LDA
dictionary = corpora.Dictionary(processed_conversations)
corpus = [dictionary.doc2bow(text) for text in processed_conversations]

# Perform LDA
num_topics = 2  # Number of topics to find
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Display the topics
topics = lda_model.print_topics(num_words=4)
for topic in topics:
    print(topic)

# Assign topics to each sentence in the conversations
def assign_topics_to_sentences(conversations, lda_model, dictionary):
    sentence_topic_mapping = []
    for conversation in conversations:
        sentences = sent_tokenize(conversation)
        processed_sentences = [preprocess(sentence) for sentence in sentences]
        bow_sentences = [dictionary.doc2bow(text) for text in processed_sentences]
        topics_per_sentence = [lda_model.get_document_topics(bow) for bow in bow_sentences]
        sentence_topic_mapping.append(list(zip(sentences, topics_per_sentence)))
    return sentence_topic_mapping

sentence_topic_mapping = assign_topics_to_sentences(conversations, lda_model, dictionary)

# Display the assigned topics for each sentence
for i, conversation_mapping in enumerate(sentence_topic_mapping):
    print(f"Conversation {i+1}:")
    for sentence, topics in conversation_mapping:
        topic = max(topics, key=lambda x: x[1])[0]  # Choose the topic with the highest probability
        print(f"  Sentence: {sentence}")
        print(f"  Assigned Topic: {topic}")

# Contextual splitting based on topic changes
def contextual_splitting(sentence_topic_mapping):
    segmented_conversations = []
    for conversation_mapping in sentence_topic_mapping:
        segments = []
        current_segment = []
        current_topic = None
        for sentence, topics in conversation_mapping:
            topic = max(topics, key=lambda x: x[1])[0]
            if current_topic is None:
                current_topic = topic
            if topic != current_topic and current_segment:
                segments.append(current_segment)
                current_segment = []
                current_topic = topic
            current_segment.append(sentence)
        if current_segment:
            segments.append(current_segment)
        segmented_conversations.append(segments)
    return segmented_conversations

segmented_conversations = contextual_splitting(sentence_topic_mapping)

# Display the segmented conversations
for i, segments in enumerate(segmented_conversations):
    print(f"Segmented Conversation {i+1}:")
    for j, segment in enumerate(segments):
        print(f"  Segment {j+1}: {' '.join(segment)}")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(0, '0.033*"customer" + 0.030*"agent" + 0.019*"network" + 0.015*"emma"')
(1, '0.024*"calls" + 0.024*"data" + 0.021*"international" + 0.021*"plan"')
Conversation 1:
  Sentence: Agent: Good morning!
  Assigned Topic: 0
  Sentence: Thank you for calling XYZ Mobile Network.
  Assigned Topic: 0
  Sentence: My name is Alex.
  Assigned Topic: 0
  Sentence: How can I assist you today?
  Assigned Topic: 0
  Sentence: Customer: Hi Alex, I'm Emma.
  Assigned Topic: 0
Conversation 2:
  Sentence: I have a few issues with my mobile network, and I was hoping you could help me out.
  Assigned Topic: 0
  Sentence: Agent: Of course, Emma.
  Assigned Topic: 0
  Sentence: I'm here to help.
  Assigned Topic: 0
Conversation 3:
  Sentence: Could you please describe the iss

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import string

# Download stopwords if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# # Sample conversations
# conversations = [
#     "This is the first conversation. It talks about sports and games. The match was exciting. However, the weather was bad.",
#     "This is another conversation. It discusses technology. Artificial intelligence is fascinating. Machine learning is a subset of AI."
# ]

# Text preprocessing function
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

# Preprocess each conversation
processed_conversations = [preprocess(conversation) for conversation in conversations]

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_conversations)

# Perform NMF
num_topics = 2  # Number of topics to find
nmf_model = NMF(n_components=num_topics, random_state=1)
nmf_features = nmf_model.fit_transform(tfidf_matrix)

# Display the topics
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"Topic {topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-4 - 1:-1]]))

# Assign topics to each sentence in the conversations
def assign_topics_to_sentences(conversations, nmf_model, vectorizer):
    sentence_topic_mapping = []
    for conversation in conversations:
        sentences = sent_tokenize(conversation)
        processed_sentences = [preprocess(sentence) for sentence in sentences]
        tfidf_sentences = vectorizer.transform(processed_sentences)
        topics_per_sentence = nmf_model.transform(tfidf_sentences)
        sentence_topic_mapping.append(list(zip(sentences, topics_per_sentence)))
    return sentence_topic_mapping

sentence_topic_mapping = assign_topics_to_sentences(conversations, nmf_model, vectorizer)

# Display the assigned topics for each sentence
for i, conversation_mapping in enumerate(sentence_topic_mapping):
    print(f"Conversation {i+1}:")
    for sentence, topics in conversation_mapping:
        topic = topics.argmax()  # Choose the topic with the highest weight
        print(f"  Sentence: {sentence}")
        print(f"  Assigned Topic: {topic}")

# Contextual splitting based on topic changes
def contextual_splitting(sentence_topic_mapping):
    segmented_conversations = []
    for conversation_mapping in sentence_topic_mapping:
        segments = []
        current_segment = []
        current_topic = None
        for sentence, topics in conversation_mapping:
            topic = topics.argmax()
            if current_topic is None:
                current_topic = topic
            if topic != current_topic and current_segment:
                segments.append(current_segment)
                current_segment = []
                current_topic = topic
            current_segment.append(sentence)
        if current_segment:
            segments.append(current_segment)
        segmented_conversations.append(segments)
    return segmented_conversations

segmented_conversations = contextual_splitting(sentence_topic_mapping)

# Display the segmented conversations
for i, segments in enumerate(segmented_conversations):
    print(f"Segmented Conversation {i+1}:")
    for j, segment in enumerate(segments):
        print(f"  Segment {j+1}: {' '.join(segment)}")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Topic 0:
data international calls using
Topic 1:
agent plan customer premium
Conversation 1:
  Sentence: Agent: Good morning!
  Assigned Topic: 1
  Sentence: Thank you for calling XYZ Mobile Network.
  Assigned Topic: 1
  Sentence: My name is Alex.
  Assigned Topic: 1
  Sentence: How can I assist you today?
  Assigned Topic: 1
  Sentence: Customer: Hi Alex, I'm Emma.
  Assigned Topic: 1
Conversation 2:
  Sentence: I have a few issues with my mobile network, and I was hoping you could help me out.
  Assigned Topic: 1
  Sentence: Agent: Of course, Emma.
  Assigned Topic: 1
  Sentence: I'm here to help.
  Assigned Topic: 1
Conversation 3:
  Sentence: Could you please describe the issues you're experiencing?
  Assigned Topic: 1
  Sentence: Customer: Well

In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.texttiling import TextTilingTokenizer

# Download necessary NLTK data if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Sample conversations with added paragraph breaks
conversations = [
    "This is the first conversation.\n\n It talks about sports and games.\n\nThe match was exciting. However, the weather was bad.\n\n"
    "Let's discuss technology now. \n\nArtificial intelligence is fascinating. Machine learning is a subset of AI.",
    "Another conversation starts here. It covers travel and adventures.\n\nTraveling can be exciting.\n\n"
    "Then, it moves to food. Food is a central part of travel experiences. Culinary adventures are the best."
]

# Text preprocessing function
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

# Preprocess each conversation
processed_conversations = [preprocess(conversation) for conversation in conversations]

# TextTiling for contextual splitting
def text_tiling_segmentation(conversations):
    tokenizer = TextTilingTokenizer()
    segmented_conversations = []
    for conversation in conversations:
        segments = tokenizer.tokenize(conversation)
        segmented_conversations.append(segments)
    return segmented_conversations

segmented_conversations = text_tiling_segmentation(processed_conversations)

# Display the segmented conversations
for i, segments in enumerate(segmented_conversations):
    print(f"Segmented Conversation {i+1}:")
    for j, segment in enumerate(segments):
        print(f"  Segment {j+1}: {segment.strip()}")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: No paragraph breaks were found(text too short perhaps?)

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
import string

# Download necessary NLTK data if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# # Sample conversations
# conversations = [
#     "This is the first conversation. It talks about sports and games. The match was exciting. However, the weather was bad. "
#     "Let's discuss technology now. Artificial intelligence is fascinating. Machine learning is a subset of AI.",
#     "Another conversation starts here. It covers travel and adventures. Traveling can be exciting. "
#     "Then, it moves to food. Food is a central part of travel experiences. Culinary adventures are the best."
# ]

# Text preprocessing function
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

# Preprocess each conversation
processed_conversations = [preprocess(conversation) for conversation in conversations]

# Bayesian Segmentation for contextual splitting
def bayesian_segmentation(conversations):
    tokenizer = PunktSentenceTokenizer()
    segmented_conversations = []
    for conversation in conversations:
        segments = tokenizer.tokenize(conversation)
        segmented_conversations.append(segments)
    return segmented_conversations

segmented_conversations = bayesian_segmentation(processed_conversations)

# Display the segmented conversations
for i, segments in enumerate(segmented_conversations):
    print(f"Segmented Conversation {i+1}:")
    for j, segment in enumerate(segments):
        print(f"  Segment {j+1}: {segment.strip()}")


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Segmented Conversation 1:
  Segment 1: agent good morning thank calling xyz mobile network name alex assist today customer hi alex emma
Segmented Conversation 2:
  Segment 1: issues mobile network hoping could help agent course emma help
Segmented Conversation 3:
  Segment 1: could please describe issues experiencing customer well firstly trouble signal
Segmented Conversation 4:
  Segment 1: seems drop unexpectedly times ca make receive calls even though shows full bars
Segmented Conversation 5:
  Segment 1: agent sorry hear experiencing signal issues emma let start checking things
Segmented Conversation 6:
  Segment 1: tell usually face problem specific location like home office happen everywhere
Segmented Conversation 7:
  Segment 1: customer mostl

In [29]:
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize

from transformers import pipeline
emotion_classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [56]:
from tqdm import tqdm

In [61]:
def preprocess(text):
    return sent_tokenize(text)

def segment_by_emotion(conversations):
    segmented_conversation = []
    prev_emotion = None
    for sentence in tqdm(conversations):
        emotion_result = emotion_classifier(preprocess(sentence))
        if(len(emotion_result)):
#             print(pd.DataFrame(emotion_result[0]))
            emotion = pd.DataFrame(emotion_classifier(preprocess(sentence))[0]).sort_values(by='score',ascending=False).reset_index(drop=True).loc[0,'label']
            if prev_emotion is None:
                prev_emotion = emotion
            elif emotion != prev_emotion:
                segmented_conversation.append("\n\n\n")  # Add a newline to indicate segment boundary
                prev_emotion = emotion
            segmented_conversation.append(sentence)
    return ' '.join(segmented_conversation)

segmented_conversation = segment_by_emotion(conversations)
print(segmented_conversation)


100%|██████████| 48/48 [00:13<00:00,  3.67it/s]

Agent: Good morning! Thank you for calling XYZ Mobile Network. My name is Alex. How can I assist you today? Customer: Hi Alex, I'm Emma. I have a few issues with my mobile network, and I was hoping you could help me out. Agent: Of course, Emma. I'm here to help. Could you please describe the issues you're experiencing? Customer: Well, firstly, I've been having trouble with my signal. 


 It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars. 


 Agent: I'm sorry to hear that you're experiencing signal issues, Emma. Let's start by checking a few things. 


 Can you tell me where you usually face this problem? Is it a specific location like your home or office, or does it happen everywhere? 


 Customer: It's mostly at home. I live in an apartment on the 10th floor, and the signal is really unstable here. Agent: I understand. 


 Sometimes, signal issues can be due to building materials or interference. Can you t

In [63]:
from textblob import TextBlob

In [65]:
#sentiment

# Preprocess the conversation by tokenizing into sentences
sentences = sent_tokenize(''.join(conversations))

# Function to perform sentiment analysis and segment based on sentiment shifts
def segment_by_sentiment(sentences):
    segmented_conversation = []
    prev_sentiment = None
    for sentence in sentences:
        blob = TextBlob(sentence)
        sentiment = blob.sentiment.polarity
        if prev_sentiment is None:
            prev_sentiment = sentiment
        elif sentiment * prev_sentiment <= 0:
            segmented_conversation.append("\n\n\n")  # Add a newline to indicate segment boundary
            prev_sentiment = sentiment
        segmented_conversation.append(sentence)
    return ' '.join(segmented_conversation)

# Segment the conversation based on sentiment shifts
segmented_conversation = segment_by_sentiment(sentences)

# Display the segmented conversation
print(segmented_conversation)

Agent: Good morning! 


 Thank you for calling XYZ Mobile Network. 


 My name is Alex. 


 How can I assist you today? 


 Customer: Hi Alex, I'm Emma.I have a few issues with my mobile network, and I was hoping you could help me out. 


 Agent: Of course, Emma. 


 I'm here to help.Could you please describe the issues you're experiencing? 


 Customer: Well, firstly, I've been having trouble with my signal.It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars.Agent: I'm sorry to hear that you're experiencing signal issues, Emma. 


 Let's start by checking a few things.Can you tell me where you usually face this problem? 


 Is it a specific location like your home or office, or does it happen everywhere?Customer: It's mostly at home. I live in an apartment on the 10th floor, and the signal is really unstable here. 


 Agent: I understand.Sometimes, signal issues can be due to building materials or interferen

In [67]:
import spacy
from spacy.lang.en import English
import nltk
from nltk.tokenize import sent_tokenize

# Download NLTK data if not already downloaded
nltk.download('punkt')

# Load the English language model for spaCy
nlp = spacy.load('en_core_web_sm')


# Preprocess the conversation by tokenizing into sentences
sentences = sent_tokenize(conversation)

# Function to perform Named Entity Recognition (NER) and segment based on new entities
def segment_by_ner(sentences):
    segmented_conversation = []
    prev_entities = set()
    for sentence in sentences:
        doc = nlp(sentence)
        entities = set([ent.text for ent in doc.ents])
        new_entities = entities - prev_entities
        if new_entities:
            segmented_conversation.append("\n\n\n")  # Add a newline to indicate segment boundary
            prev_entities.update(entities)
        segmented_conversation.append(sentence)
    return ' '.join(segmented_conversation)

# Segment the conversation based on NER
segmented_conversation = segment_by_ner(sentences)

# Display the segmented conversation
print(segmented_conversation)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



 Agent: Good morning! 


 Thank you for calling XYZ Mobile Network. 


 My name is Alex. 


 How can I assist you today? 


 Customer: Hi Alex, I'm Emma.I have a few issues with my mobile network, and I was hoping you could help me out. Agent: Of course, Emma. I'm here to help.Could you please describe the issues you're experiencing? Customer: Well, firstly, I've been having trouble with my signal.It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars.Agent: I'm sorry to hear that you're experiencing signal issues, Emma. Let's start by checking a few things.Can you tell me where you usually face this problem? Is it a specific location like your home or office, or does it happen everywhere?Customer: It's mostly at home. 


 I live in an apartment on the 10th floor, and the signal is really unstab

In [70]:
from transformers import BertTokenizer, BertModel
import torch
import nltk
from nltk.tokenize import sent_tokenize

# Download NLTK data if not already downloaded
nltk.download('punkt')

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# # Sample conversation
# conversation = """
# This is a sample conversation. It starts positively.
# Then, it turns negative due to some issues.
# However, it ends on a positive note with a resolution.
# """

# Preprocess the conversation by tokenizing into sentences
sentences = sent_tokenize(conversation)

# Function to calculate BERT embeddings for each sentence
def get_bert_embeddings(sentences):
    # Tokenize the sentences and convert to input tensors
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)
    
    # Pass the inputs through the BERT model to get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the embeddings for each sentence
    sentence_embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Using [CLS] token embeddings
    
    return sentence_embeddings

# Function to segment the conversation based on context shifts using BERT embeddings
def segment_by_context_shift(sentences, embeddings):
    segmented_conversation = []
    prev_embedding = None
    for sentence, embedding in zip(sentences, embeddings):
        if prev_embedding is None:
            prev_embedding = embedding
        elif not torch.equal(torch.tensor(embedding), torch.tensor(prev_embedding)):
            segmented_conversation.append("\n\n\n")  # Add a newline to indicate segment boundary
            prev_embedding = embedding
        segmented_conversation.append(sentence)
    return ' '.join(segmented_conversation)

# Get BERT embeddings for each sentence
embeddings = get_bert_embeddings(sentences)

# Segment the conversation based on context shifts using BERT embeddings
segmented_conversation = segment_by_context_shift(sentences, embeddings)

# Display the segmented conversation
print(segmented_conversation)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Agent: Good morning! 


 Thank you for calling XYZ Mobile Network. 


 My name is Alex. 


 How can I assist you today? 


 Customer: Hi Alex, I'm Emma.I have a few issues with my mobile network, and I was hoping you could help me out. 


 Agent: Of course, Emma. 


 I'm here to help.Could you please describe the issues you're experiencing? 


 Customer: Well, firstly, I've been having trouble with my signal.It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars.Agent: I'm sorry to hear that you're experiencing signal issues, Emma. 


 Let's start by checking a few things.Can you tell me where you usually face this problem? 


 Is it a specific location like your home or office, or does it happen everywhere?Customer: It's mostly at home. 


 I live in an apartment on the 10th floor, and the signal 

In [74]:
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# Download NLTK data if not already downloaded
nltk.download('punkt')

# # Sample conversation
# conversation = """
# This is a sample conversation. It starts with a question.
# Then, there are some answers and greetings.
# Finally, it ends with a farewell.
# """
conversation = ''.join(conversations)
# Preprocess the conversation by tokenizing into sentences
sentences = sent_tokenize(conversation)

# Define a simple dialogue act classifier using SVC
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', SVC(kernel='linear'))
])

# Sample labeled data for training the classifier (you should have more labeled data for better accuracy)
training_data = {
    'Question': ['Is this a question?', 'What are you doing?'],
    'Answer': ['Yes, it is.', 'I am working.'],
    'Greeting': ['Hello!', 'Hi there!'],
    'Farewell': ['Goodbye.', 'See you later.']
}

# Flatten the training data into lists of sentences and corresponding labels
sentences_train = [sentence for act, sentences in training_data.items() for sentence in sentences]
labels_train = [act for act, sentences in training_data.items() for _ in sentences]

# Fit the classifier using the training data
classifier.fit(sentences_train, labels_train)

# Function to segment the conversation based on dialogue act changes
def segment_by_dialogue_act(sentences, classifier):
    segmented_conversation = []
    prev_act = None
    for sentence in sentences:
        act = classifier.predict([sentence])[0]
        if prev_act is None:
            prev_act = act
        elif act != prev_act:
            segmented_conversation.append("\n\n\n")  # Add a newline to indicate segment boundary
            prev_act = act
        segmented_conversation.append(sentence)
    return ' '.join(segmented_conversation)

# Segment the conversation based on dialogue act changes
segmented_conversation = segment_by_dialogue_act(sentences, classifier)

# Display the segmented conversation
print(segmented_conversation)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Agent: Good morning! 


 Thank you for calling XYZ Mobile Network. 


 My name is Alex. 


 How can I assist you today? 


 Customer: Hi Alex, I'm Emma.I have a few issues with my mobile network, and I was hoping you could help me out. Agent: Of course, Emma. 


 I'm here to help.Could you please describe the issues you're experiencing? 


 Customer: Well, firstly, I've been having trouble with my signal.It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars.Agent: I'm sorry to hear that you're experiencing signal issues, Emma. 


 Let's start by checking a few things.Can you tell me where you usually face this problem? 


 Is it a specific location like your home or office, or does it happen everywhere?Customer: It's mostly at home. I live in an apartment on the 10th floor, and the signal is reall

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
import nltk
from nltk.tokenize import sent_tokenize

# Download NLTK data if not already downloaded
nltk.download('punkt')


# Preprocess the conversation by tokenizing into sentences
sentences = sent_tokenize(conversation)

# Text preprocessing function
def preprocess(text):
    return text.lower()

# Preprocess each sentence
preprocessed_sentences = [preprocess(sentence) for sentence in sentences]

# Vectorize the sentences using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_sentences)

# Apply hierarchical clustering
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5, linkage='ward')
clustering.fit(X.toarray())

# Function to segment the conversation based on hierarchical clustering
def segment_by_clustering(sentences, labels):
    segmented_conversation = []
    prev_label = None
    for sentence, label in zip(sentences, labels):
        if prev_label is None:
            prev_label = label
        elif label != prev_label:
            segmented_conversation.append("\n\n\n")  # Add a newline to indicate segment boundary
            prev_label = label
        segmented_conversation.append(sentence)
    return ' '.join(segmented_conversation)

# Segment the conversation based on hierarchical clustering
segmented_conversation = segment_by_clustering(sentences, clustering.labels_)

# Display the segmented conversation
print(segmented_conversation)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Agent: Good morning! 


 Thank you for calling XYZ Mobile Network. 


 My name is Alex. 


 How can I assist you today? 


 Customer: Hi Alex, I'm Emma.I have a few issues with my mobile network, and I was hoping you could help me out. 


 Agent: Of course, Emma. 


 I'm here to help.Could you please describe the issues you're experiencing? 


 Customer: Well, firstly, I've been having trouble with my signal.It seems to drop unexpectedly, and there are times when I can't make or receive calls, even though it shows that I have full bars.Agent: I'm sorry to hear that you're experiencing signal issues, Emma. 


 Let's start by checking a few things.Can you tell me where you usually face this problem? 


 Is it a specific location like your home or office, or does it happen everywhere?Customer: It's mostly at home. 


 I live in an apartment on the 10th floor, and the signal 

In [ ]:
nltk.download()

In [77]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK data if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


# Preprocess the conversation by tokenizing into sentences and performing text normalization
def preprocess(text):
    sentences = sent_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    preprocessed_sentences = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence.lower())
        words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
        preprocessed_sentences.append(' '.join(words))
    return preprocessed_sentences

# Calculate TF-IDF vectors for the preprocessed sentences
preprocessed_sentences = preprocess(conversation)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_sentences)

# Calculate cosine similarity between consecutive sentences
def calculate_similarity(matrix):
    similarities = []
    for i in range(1, matrix.shape[0]):
        similarity = cosine_similarity(matrix[i-1], matrix[i])[0][0]
        similarities.append(similarity)
    return similarities

# Function to segment the conversation based on lexical coherence
def segment_by_lexical_coherence(sentences, similarities, threshold=0.5):
    segmented_conversation = []
    prev_similarity = None
    for sentence, similarity in zip(sentences, similarities):
        if prev_similarity is None:
            prev_similarity = similarity
        elif similarity < threshold:
            segmented_conversation.append("\n")  # Add a newline to indicate segment boundary
            prev_similarity = similarity
        segmented_conversation.append(sentence)
    return ' '.join(segmented_conversation)

# Segment the conversation based on lexical coherence
similarities = calculate_similarity(tfidf_matrix)
segmented_conversation = segment_by_lexical_coherence(preprocessed_sentences, similarities)

# Display the segmented conversation
print(segmented_conversation)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

# Download NLTK data if not already downloaded
nltk.download('punkt')

# # Sample conversation
# conversation = """
# This is a sample conversation. It starts with an introduction.
# Then, there are some details in the body of the conversation.
# Finally, it ends with a conclusion.
# """

# Preprocess the conversation by tokenizing into sentences
sentences = sent_tokenize(conversation)

# Calculate TF-IDF vectors for the sentences
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences)

# Calculate cosine similarity between sentences
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Build a graph with sentences as nodes and cosine similarity as edge weights
graph = nx.Graph()
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        graph.add_edge(i, j, weight=cosine_similarities[i][j])

# Use graph partitioning algorithms (e.g., Louvain) to identify contextually coherent segments
partitions = nx.community.greedy_modularity_communities(graph)

# Function to segment the conversation based on identified partitions
def segment_by_graph_partitions(sentences, partitions):
    segmented_conversation = []
    for partition in partitions:
        segment = ' '.join([sentences[node] for node in partition])
        segmented_conversation.append(segment)
    return segmented_conversation

# Segment the conversation based on graph partitions
segmented_conversation = segment_by_graph_partitions(sentences, partitions)

# Display the segmented conversation
print('\n'.join(segmented_conversation))
